In [21]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [24]:
!pip install deltalake

   ---------------------------------------- 0.0/47.9 MB ? eta -:--:--
   --------- ------------------------------ 11.0/47.9 MB 76.8 MB/s eta 0:00:01
   --------------------------- ------------ 33.0/47.9 MB 91.3 MB/s eta 0:00:01
   ---------------------------------------  47.7/47.9 MB 98.0 MB/s eta 0:00:01
   ---------------------------------------  47.7/47.9 MB 98.0 MB/s eta 0:00:01
   ---------------------------------------- 47.9/47.9 MB 52.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 2.9/2.9 MB 80.9 MB/s eta 0:00:00


In [25]:
# ---------- Step 1: Import Libraries ----------
import pandas as pd
import numpy as np
import gdown
import os
from deltalake import write_deltalake

# ---------- Step 2: Download Raw Dataset from Google Drive ----------
# File is stored in Google Drive (public share link)
file_id = "11OcLKGiakMl2cP3Dz4gVh31T2R01enPk"
raw_path = "DineSafe.csv"

# Download the dataset if not already present
if not os.path.exists(raw_path):
    print("⬇️ Downloading DineSafe dataset from Google Drive...")
    gdown.download(id=file_id, output=raw_path, quiet=False)
else:
    print("✅ Found existing local file. Skipping download.")

# ---------- Step 3: Load Dataset ----------
print("\n📂 Loading dataset...")
df = pd.read_csv(raw_path)
print(f"✅ Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns\n")

# ---------- Step 4: Initial Overview ----------
print("🔍 Preview of raw data:")
print(df.head(), "\n")
print("Missing values per column:")
print(df.isnull().sum(), "\n")

# ---------- Step 5: Handle Missing Values ----------
print("🔧 Handling missing values...")

# Delete records with missing key fields
df = df.dropna(subset=["Inspection ID", "Inspection Date", "Establishment Name"])

#  Fill in the missing logically
df["Infraction Details"] = df["Infraction Details"].fillna("No Infraction")
df["Severity"] = df["Severity"].fillna("No Infraction")
df["Action"] = df["Action"].fillna("None")
df["Outcome"] = df["Outcome"].fillna("Pending/No Action")
df["Amount Fined"] = df["Amount Fined"].fillna(0)

print("✅ Missing values handled.\n")
print(df.isnull().sum())


# ---------- Step 6: Remove Duplicates ----------

dup_count = df.duplicated().sum()
print(f"🔍 Found {dup_count} duplicate rows before removal.")

before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]

print(f"✅ Removed {before - after} duplicate rows.\n")


# ---------- Step 7: Correct Data Types ----------
# Convert date columns
for col in df.columns:
    if 'Date' in col:
        try:
            df[col] = pd.to_datetime(df[col], errors='coerce')
        except Exception:
            pass

print("✅ Data types standardized.\n")

# ---------- Step 8: Clean Text Fields ----------
text_cols = ['Establishment Name', 'Address', 'City', 'Infraction Details']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].str.strip().str.title()

print("✅ Text fields cleaned.\n")

# ---------- Step 9: Export Cleaned Dataset ----------
clean_dir = "data_cleaned"
os.makedirs(clean_dir, exist_ok=True)
clean_path = os.path.join(clean_dir, "DineSafe_Cleaned.csv")
df.to_csv(clean_path, index=False)

print(f"💾 Cleaned dataset saved to: {clean_path}")
print(f"✅ Final shape: {df.shape[0]} rows, {df.shape[1]} columns")

#  Export as Parquet file
parquet_path = os.path.join(clean_dir, "DineSafe_Cleaned.parquet")
df.to_parquet(parquet_path, index=False)
print(f"💾 Parquet file saved to: {parquet_path}")

#  Export as Delta Lake table (optional, for reproducibility)
delta_dir = os.path.join(clean_dir, "DineSafe_Delta")
write_deltalake(delta_dir, df)
print(f"💾 Delta Lake versioned dataset created at: {delta_dir}")

#  Final summary
print(f"✅ Final shape: {df.shape[0]} rows × {df.shape[1]} columns")


✅ Found existing local file. Skipping download.

📂 Loading dataset...
✅ Dataset loaded: 124740 rows, 17 columns

🔍 Preview of raw data:
   _id  Establishment ID  Inspection ID          Establishment Name  \
0    1          10657713    105133203.0       NEW KANTAMANTO MARKET   
1    2          10657713    105133203.0       NEW KANTAMANTO MARKET   
2    3          10657713    105238109.0       NEW KANTAMANTO MARKET   
3   10          10752656    105149282.0  # HASHTAG INDIA RESTAURANT   
4   11          10752656    105149282.0  # HASHTAG INDIA RESTAURANT   

  Establishment Type      Establishment Address Establishment Status  \
0         Food Depot  266 EDDYSTONE AVE, Unit-0                 Pass   
1         Food Depot  266 EDDYSTONE AVE, Unit-0                 Pass   
2         Food Depot  266 EDDYSTONE AVE, Unit-0                 Pass   
3      Food Take Out           1871 O'CONNOR DR                 Pass   
4      Food Take Out           1871 O'CONNOR DR                 Pass   

  Mi